# **Requirements**

Python 3.8.10

pythonnet 2.5.2

`pip install pythonnet==2.5.2`

In [142]:
from classes.base_interactive import PythonZOSConnection
from classes.beam_shaper import BeamShaperFitter, BeamShaper, EvenAsphere
import sys
from importlib import metadata
import numpy as np

print("Python version: ", sys.version)
print('Pythonnet version:', metadata.version('pythonnet'))

Python version:  3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pythonnet version: 2.5.2


# **Connect to Zemax and initiate a system**

In [143]:
zos = PythonZOSConnection()
ZOSAPI = zos.ZOSAPI
TheApplication = zos.TheApplication
TheSystem = zos.TheSystem

# TheSystem.Mode = ZOSAPI.ZOSAPI_Mode.Plugin
TheSystem.UpdateMode = ZOSAPI.LensUpdateMode.AllWindows

print('Connected to OpticStudio')
# The connection should now be ready to use.  For example:
print('Serial #: ', TheApplication.SerialCode)

print('ZOSAPI Mode: ', TheSystem.Mode)
print("Lens update mode: ", TheSystem.UpdateMode)

SysExplore = TheSystem.SystemData
SysLDE = TheSystem.LDE
TheMFE = TheSystem.MFE

Connected to OpticStudio
Serial #:  20120530
ZOSAPI Mode:  0
Lens update mode:  2


In [144]:
galilean = BeamShaper(omega_0 = 2.366,
                  R_max=12.15,
                  r_max=4.05,
                  d=150,
                  n=1.46071,
                  type='Galilean'
                )

print("Vertex_radius_1:", galilean.r_c1)
print("Conic_constant_1:", galilean.k_1)
print("Vertex_radius_2:", galilean.r_c2)
print("Conic_constant_2:", galilean.k_2)


Vertex_radius_1: 11.017003283425508
Conic_constant_1: -30.692685297894425
Vertex_radius_2: 80.12350328342549
Conic_constant_2: -6.060545490862053


In [145]:
galileanfitter = BeamShaperFitter(galilean, num_samples=1000)

ea1 = EvenAsphere(r_max=galilean.r_max, vertex_r=galilean.r_c1, k=galilean.k_1, coefficients={}, num_samples=galileanfitter.num_samples)
ea2 = EvenAsphere(r_max=galilean.R_max, vertex_r=galilean.r_c2, k=galilean.k_2, coefficients={}, num_samples=galileanfitter.num_samples)

sag_fit = galileanfitter.ls_fit(ea1 = ea1, ea2 = ea2)

sag_fit_s1 = sag_fit['surface1']
sag_fit_s2 = sag_fit['surface2']
print("sag fitting surface 1 vertex_radius=", sag_fit_s1['vertex_r'])
print("sag fitting surface 1 conic_constant=", sag_fit_s1['k'])

print("sag fitting surface 2 vertex_radius=", sag_fit_s2['vertex_r'])
print("sag fitting surface 2 conic_constant=", sag_fit_s2['k'])

print("rms", sag_fit['rms'])

sag fitting surface 1 vertex_radius= 9.781372027845226
sag fitting surface 1 conic_constant= -49.50164083881997
sag fitting surface 2 vertex_radius= 78.88787202784522
sag fitting surface 2 conic_constant= -11.604267862034117
rms 0.0019703085099571052


In [146]:
dsag_fit = galileanfitter.ls_fit(ea1 = ea1, ea2 = ea2, fit_type='grad_sag')

dsag_fit_s1 = dsag_fit['surface1']
dsag_fit_s2 = dsag_fit['surface2']
print("dsag fitting surface 1 vertex_radius=", dsag_fit_s1['vertex_r'])
print("dsag fitting surface 1 conic_constant=", dsag_fit_s1['k'])

print("dsag fitting surface 2 vertex_radius=", dsag_fit_s2['vertex_r'])
print("dsag fitting surface 2 conic_constant=", dsag_fit_s2['k'])

print("rms", dsag_fit['rms'])

dsag fitting surface 1 vertex_radius= 9.219229221283971
dsag fitting surface 1 conic_constant= -54.30293019107848
dsag fitting surface 2 vertex_radius= 78.32572922128396
dsag fitting surface 2 conic_constant= -13.995890422984257
rms 0.004474569797059773


# **Set System Data**

In [147]:
# Control variable for this chapter
RUN_SYSTEM_DATA = True  # Set to False to skip system data configuration

## **Set Title and Notes**

In [148]:
if RUN_SYSTEM_DATA:
    # Set Title and Notes
    SysExplore.TitleNotes.Title = "Galilean Beam Shaper"
    SysExplore.TitleNotes.Notes = "Galilean beam shaper design, turning a Gaussian into a flat-top beam."
    SysExplore.TitleNotes.Author = "Ziyi Xiong"

## **Set Aperture**

In [149]:
if RUN_SYSTEM_DATA:
    # Set Aperture
    SysExplore.Aperture.ApertureType = ZOSAPI.SystemData.ZemaxApertureType.EntrancePupilDiameter
    SysExplore.Aperture.ApertureValue = galilean.r_max * 2

    SysExplore.Aperture.ApodizationType = 1 # 0: Uniform, 1: Gaussian, 2: Cosine Cubed
    SysExplore.Aperture.ApodizationFactor = galilean.apodization_factor # Apodization factor for Gaussian

## **Set Fields**

In [150]:
if RUN_SYSTEM_DATA:
    # Set Fields
    SysExplore.Fields.SetFieldType(ZOSAPI.SystemData.FieldType.ParaxialImageHeight)
    #SysExplore.Fields.ApplyFieldWizard(ZOSAPI.SystemData.FieldPattern.EqualAreaY, 9, 6.6, 0, 0, 0, True, False)

## **Set Wavelengths**

### **Via Presets**

In [151]:
# SysExplore.Wavelengths.SelectWavelengthPreset(ZOSAPI.SystemData.WavelengthPreset.FdC_Visible)

### **Customize Wavelengths**

In [152]:
if RUN_SYSTEM_DATA:
    SysExplore.Wavelengths.GetWavelength(1).Wavelength = 0.532

### **Remove Wavelengths**

In [153]:
# if num_wavelengths > 1: [SysExplore.Wavelengths.RemoveWavelength(i) for i in range(num_wavelengths, 1, -1)]

# **Set Lens Data**

In [154]:
# Control variable for this chapter
RUN_LENS_DATA = True  # Set to False to skip lens data configuration

## **Add Surfaces**

In [155]:
if RUN_LENS_DATA:
    num_surfaces = SysLDE.NumberOfSurfaces

    if num_surfaces == 3:
        for i in range(5): # range(5) = [0, 1, 2, 3, 4]
            SysLDE.AddSurface()

    num_surfaces = SysLDE.NumberOfSurfaces
    print("Number of surfaces after insertion: ", num_surfaces)

Number of surfaces after insertion:  8


## **Get and Set Surfaces**

In [156]:
if RUN_LENS_DATA:
    # Use a list to store all surfaces objects
    Surface=[SysLDE.GetSurfaceAt(i) for i in range(0, num_surfaces)] 

## **Set Surface Data**

### **Set Stop**

In [157]:
if RUN_LENS_DATA:
    Surface[2].IsStop = True 

### **Set Lens Data**

In [ ]:
if RUN_LENS_DATA:
    Surface[1].Thickness = 10
    Surface[1].Comment = "Dummy"

    Surface[2].Thickness = 3
    MaterialModel1 = Surface[2].MaterialCell.CreateSolveType(ZOSAPI.Editors.SolveType.MaterialModel)._S_MaterialModel
    MaterialModel1.IndexNd = galilean.n
    Surface[2].MaterialCell.SetSolveData(MaterialModel1)
    Surface[2].SemiDiameter = galilean.r_max
    Surface[2].MechanicalSemiDiameter = galilean.r_max

    Surface[3].Thickness = galilean.d
    Surface[3].Radius = dsag_fit_s1['vertex_r']
    Surface[3].Conic = dsag_fit_s1['k']
    Surface[3].SemiDiameter = galilean.r_max
    Surface[3].MechanicalSemiDiameter = galilean.r_max

    Surface[5].Thickness = 3
    MaterialModel2 = Surface[5].MaterialCell.CreateSolveType(ZOSAPI.Editors.SolveType.MaterialModel)._S_MaterialModel
    MaterialModel2.IndexNd = galilean.n
    Surface[5].MaterialCell.SetSolveData(MaterialModel2)
    Surface[5].Radius = dsag_fit_s2['vertex_r']
    Surface[5].Conic = dsag_fit_s2['k']
    
    Surface[6].Thickness = 10

# **Optimization**

In [159]:
# Control variable for this chapter
RUN_OPTIMIZATION = True  # Set to True to enable optimization

## **Set Variables**

In [160]:
if RUN_OPTIMIZATION:
    Surface[3].RadiusCell.MakeSolveVariable()
    Surface[3].ConicCell.MakeSolveVariable()
    Surface[5].RadiusCell.MakeSolveVariable()
    Surface[5].ConicCell.MakeSolveVariable()
else:
    print("Optimization chapter skipped.")

## **Set Operands**

In [161]:
if RUN_OPTIMIZATION:
    # help(TheMFE)

    num_operands = TheMFE.NumberOfOperands
    # TheMFE.RemoveOperandsAt(1, num_operands)

In [162]:
if RUN_OPTIMIZATION:
    num_operands = TheMFE.NumberOfOperands

    num_samples = 81 # best number of sampling points found for conic constants only optimization

    # num_samples = 101
    if num_operands == 1:
        for i in range(2 * num_samples):
            TheMFE.AddOperand()

    num_operands = TheMFE.NumberOfOperands

    print(num_operands)

    REAY = ZOSAPI.Editors.MFE.MeritOperandType.REAY
    REAB = ZOSAPI.Editors.MFE.MeritOperandType.REAB

163


In [163]:
if RUN_OPTIMIZATION:
    Operand_REAY = [TheMFE.GetOperandAt(i) for i in range(1, num_samples + 1)]

In [164]:
if RUN_OPTIMIZATION:
    for i in range(len(Operand_REAY)):
        Operand_REAY[i].ChangeType(REAY)
        Operand_REAY[i].GetCellAt(2).IntegerValue = 4
        Operand_REAY[i].Weight = 50.0
        py = i / (num_samples - 1) # uniform sampling ensuring sharpe edges
        Operand_REAY[i].GetCellAt(7).DoubleValue = py
        r = galilean.r_max * py
        R = galilean.R_max * np.sqrt(1 - np.exp(-2 * (r / galilean.omega_0)**2)) / np.sqrt(1 - np.exp(-2 * (galilean.r_max / galilean.omega_0)**2))
        Operand_REAY[i].Target = R

In [165]:
if RUN_OPTIMIZATION:
    Operand_REAB = [TheMFE.GetOperandAt(i) for i in range(num_samples + 1, num_operands)]
    for i in range(len(Operand_REAB)):
        Operand_REAB[i].ChangeType(REAB)
        Operand_REAB[i].GetCellAt(2).IntegerValue = 7 # Nummber of surface
        Operand_REAB[i].Weight = 1.0
        py = i / (num_samples - 1)
        Operand_REAB[i].GetCellAt(7).DoubleValue = py
        Operand_REAB[i].Target = 0.0

In [166]:
if RUN_OPTIMIZATION:
    TheMFE.CalculateMeritFunction()

Possible improvements:

1. Greater weights around center
2. Adjust Weights based on error
3. More sampling around the center??

In [ ]:
'''
Only optimize the flat part 

For Galilean beam shaper, it seems impossible to have a flat top and wanted beam size at the same time when only  values are used.
'''
if RUN_OPTIMIZATION:
    for i in range(0, 25):
        Operand_REAY[i].Weight = np.abs(Operand_REAY[i].Target - Operand_REAY[i].Value) * 500.0 # adjust weights based on current error
 
    for i in range(25, len(Operand_REAY)):
        Operand_REAY[i].Weight = np.abs(Operand_REAY[i].Target - Operand_REAY[i].Value) * 0.0

In [168]:
if RUN_OPTIMIZATION:
    EFFL = ZOSAPI.Editors.MFE.MeritOperandType.EFFL
    # TheMFE.AddOperand()
    Operand_EFFL = TheMFE.GetOperandAt(num_operands)
    Operand_EFFL.ChangeType(EFFL)
    Operand_EFFL.Target = 1e+10
    Operand_EFFL.Weight = 1

## **Optimize**

if RUN_OPTIMIZATION:
    run_localopt = input("Run local optimization? (y/n):")
    LocalOpt = TheSystem.Tools.OpenLocalOptimization()
    if run_localopt.lower() == 'y':
        LocalOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.DampedLeastSquares
        LocalOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Automatic
        LocalOpt.NumberOfCores = 16
        LocalOpt.RunAndWaitForCompletion()
        LocalOpt.Close()
    else:
        print("Optimization skipped.")

if RUN_OPTIMIZATION:
    run_hammeropt = input("Run Hammer optimization? (y/n):")
    HammerOpt = TheSystem.Tools.OpenHammerOptimization()
    if run_hammeropt.lower() == 'y':
        HammerOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.DampedLeastSquares
        HammerOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Automatic
        HammerOpt.NumberOfCores = 16
        HammerOpt.RunAndWaitForCompletion()
        HammerOpt.Close()
    else:
        print("Hammer optimization skipped.")

if RUN_OPTIMIZATION:
    HammerOpt = TheSystem.Tools.OpenHammerOptimization()
    help(TheSystem.Tools.OpenLocalOptimization())

if RUN_OPTIMIZATION:
    HammerOpt.MaxCores()

# **Analysis**

In [169]:
# Control variable for this chapter
RUN_ANALYSIS = False  # Set to False to skip analysis

In [170]:
if RUN_ANALYSIS:
    TheAna = TheSystem.Analyses
    num_analyses = TheAna.NumberOfAnalyses
    print(num_analyses)
    #help(TheAna)

In [171]:
if RUN_ANALYSIS:
    for i in range(1, num_analyses+1): 
        ana = TheAna.Get_AnalysisAtIndex(i)
        print(f"Analysis {i}: {ana.GetAnalysisName}")

In [172]:
if RUN_ANALYSIS:
    pass
    #TheAna.CloseAnalysis(1)

    #TheAna.CloseAnalysis(2)    #TheAna.New_GeometricImageAnalysis()

In [173]:
if RUN_ANALYSIS:
    GeoImgAna1 = TheAna.Get_AnalysisAtIndex(3)
    GeoImgAna1.GetAnalysisName
    #help(GeoImgAna)

## **Set and Apply Analyses**

In [174]:
if RUN_ANALYSIS:
    GIA_Settings1 = GeoImgAna1.GetSettings()
    help(GIA_Settings1)

In [175]:
if RUN_ANALYSIS:
    GIA_Settings1.Surface.SetSurfaceNumber(-1)
    print(GIA_Settings1.Surface.GetSurfaceNumber())
    #help(GIA_Settings1.Surface)

In [176]:
if RUN_ANALYSIS:
    print(GIA_Settings1.RaysX1000)
    GIA_Settings1.RaysX1000 = 500000
    #help(GIA_Settings1.RaysX1000)

In [177]:
if RUN_ANALYSIS:
    print(GIA_Settings1.ImageSize)
    GIA_Settings1.ImageSize = 15.0
    #help(GIA_Settings1.ImageSize)

In [178]:
if RUN_ANALYSIS:
    # ShowAsTypes: Surface, Contour, GreyScale, InverseGreyScale, FalseColor, InverseFalseColor, SpotDiagram, CrossX, CrossY
    GIA_Settings1.ShowAs = ZOSAPI.Analysis.GiaShowAsTypes.CrossX
    GIA_Settings1.get_ShowAs()

In [179]:
if RUN_ANALYSIS:
    GIA_Settings1.NumberOfPixels = 200

In [180]:
if RUN_ANALYSIS:
    # Doesn't work
    GIA_Settings1.UpdateUI

In [181]:
if RUN_ANALYSIS:
    GeoImgAna1.ApplyAndWaitForCompletion()

In [182]:
#GeoImgAna.Terminate()

## **Results**

In [183]:
if RUN_ANALYSIS:
    GIA_Results1 = GeoImgAna1.GetResults()
    #help(GIA_Results)

In [184]:
if RUN_ANALYSIS:
    print(GIA_Results1.NumberOfDataGrids)
    data1 = GIA_Results1.DataGrids[0]
    #help(data1)

In [185]:
if RUN_ANALYSIS:
    #help(data1.Values)
    print(data1.Values.Length)
    print("x:", data1.Values.GetLength(0))
    print("y:", data1.Values.GetLength(1))
    zos.gia_plots(data1.Values, data1.Values.GetLength(0), data1.Values.GetLength(1), transpose=False)